In [2]:
import os
os.chdir("../scripts/")

In [3]:
import cv2
import numpy as np
import rosbag
import yaml
import data_conversion

In [4]:
DATA_PATH = "/scratchdata/processed/indoor_short"

# Open bag file
bag_file_path = "/scratchdata/indoor_short.bag"
bag = rosbag.Bag(bag_file_path)

config = yaml.load(open("../config/gemini2L.yaml", "r"), Loader=yaml.FullLoader)

In [5]:
if os.path.exists(DATA_PATH):
    os.system('rm -r {}'.format(DATA_PATH))
    
os.system('mkdir {}'.format(DATA_PATH))
os.system('mkdir {}'.format(DATA_PATH+'/rgb'))
os.system('mkdir {}'.format(DATA_PATH+'/depth'))


0

In [6]:
rgb = []

for topic, msg, t in bag.read_messages(topics=["/camera/color/image_raw"]):
    rgb.append(msg)
    prev_img = data_conversion.topic_to_image(msg)

depth = []

for topic, msg, t in bag.read_messages(topics=["/camera/depth/image_raw"]):
    depth.append(msg)


In [8]:
pt = 0
diff = 1e-5 # 0.1 ms

cnt = 0

for i in range(len(rgb)):
    rgb_msg, rgb_t = rgb[i], rgb[i].header.stamp
    depth_msg, depth_t = depth[pt], depth[pt].header.stamp
    
    while depth_t.nsecs - rgb_t.nsecs < diff * 1e9 and pt < len(depth):
        depth_msg, depth_t = depth[pt], depth[pt].header.stamp

        if abs(rgb_t.nsecs - depth_t.nsecs) < diff:
            pt += 1
            break
            
        pt += 1

    rgb_path = DATA_PATH + '/rgb/' + str(cnt) + '.png'
    depth_path = DATA_PATH + '/depth/' + str(cnt) + '.png'

    rgb_img = data_conversion.topic_to_image(rgb_msg)
    rgb_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2RGB)
    depth_img = data_conversion.topic_to_depth(depth_msg,config["depth_anything_config"])

    cv2.imwrite(rgb_path, rgb_img)
    cv2.imwrite(depth_path, depth_img)
    
    #with open(rgb_path, 'wb') as f:
    #    f.write(rgb_msg.data)
    
    #with open(depth_path, 'wb') as f:
    #    f.write(depth_msg.data)
    
    cnt += 1    
